# Visualize data on an interactive map

This notebook uses `lonboard` for interactive visualisation of data.

In [1]:
import geopandas as gpd
import numpy as np
import lonboard
from core.utils import used_keys
from lonboard.colormap import apply_continuous_cmap
import matplotlib as mpl
from mapclassify import classify
from sidecar import Sidecar


Define data path

In [2]:
chars_dir = "/data/uscuni-ulce/processed_data/chars/"

Define region

In [3]:
region = 69333

## Buildings
Load building data and ensure the geometries are all valid Polygons.

In [4]:
buildings = gpd.read_parquet(f"{chars_dir}buildings_chars_{region}.parquet").to_crs(4326).reset_index()

buildings.geometry = buildings.make_valid()

buildings = buildings[buildings.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [5]:
%%time
layer = lonboard.PolygonLayer.from_geopandas(buildings, opacity=.3)

CPU times: user 1.64 s, sys: 196 ms, total: 1.84 s
Wall time: 1.83 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [6]:
sc = Sidecar(title='buildings')

Create a Map object

In [7]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [8]:
with sc:
    display(m)

List avaialable columns

In [9]:
buildings.columns

Index(['level_0', 'index', 'currentUse', 'buildingNature', 'heightAboveGround',
       'iid', 'geometry', 'ssbCCo', 'ssbCor', 'ssbSqu', 'ssbCCM', 'ssbCCD',
       'sdbAre', 'sdbPer', 'sdbCoA', 'ssbERI', 'ssbElo', 'stbOri', 'mtbSWR',
       'libNCo', 'ldbPWL', 'ltcBuA', 'mtbAli', 'mtbNDi', 'ltbIBD', 'stbCeA',
       'nID', 'stbSAl', 'nodeID'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [10]:
# buildings.explore()

In [11]:
column = 'mtbSWR'
used_keys[column]

'shared walls ratio of buildings'

In [12]:
classifier = classify(buildings[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/mapclassify/classifiers.py:1653: UserWarning: Not enough unique values in array to form 20 classes. Setting k to 7.
  self.bins = quantile(y, k=k)


In [13]:
buildings[column].describe().iloc[1:]

mean    0.096114
std     0.188140
min     0.000000
25%     0.000000
50%     0.000000
75%     0.129638
max     1.000000
Name: mtbSWR, dtype: float64

## Tessellation

Load tessellation data and ensure the geometries are all valid Polygons.

In [15]:
tess = gpd.read_parquet(f"{chars_dir}tessellations_chars_{region}.parquet").to_crs(4326)

tess.geometry = tess.make_valid()

tess = tess[tess.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [16]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(tess, opacity=.2)

CPU times: user 2.36 s, sys: 278 ms, total: 2.64 s
Wall time: 2.63 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [17]:
sc = Sidecar(title='tess')

Create a Map object

In [18]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [19]:
with sc:
    display(m)

List avaialable columns

In [20]:
tess.columns

Index(['enclosure_index', 'geometry', 'stcOri', 'sdcLAL', 'sdcAre', 'sscCCo',
       'sscERI', 'mtcWNe', 'mdcAre', 'ltcWRB', 'sicCAR', 'stcSAl', 'nID',
       'nodeID'],
      dtype='object')

In [21]:
column = 'mtcWNe'
used_keys[column]

'perimeter-weighted neighbours of ETC'

Specify a column and pass its values into a choropleth representation within the map. 

In [22]:

classifier = classify(tess[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Enclosures

Load data and ensure the geometries are all valid Polygons.

In [35]:
enc = gpd.read_parquet(f"{chars_dir}enclosures_chars_{region}.parquet").to_crs(4326)

enc.geometry = enc.make_valid()

enc = enc[enc.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [36]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(enc, opacity=.3)

CPU times: user 168 ms, sys: 31.9 ms, total: 200 ms
Wall time: 200 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [37]:
sc = Sidecar(title='enclosures')

Create a Map object

In [38]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [39]:
with sc:
    display(m)

List avaialable columns

In [40]:
enc.columns

Index(['eID', 'geometry', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA',
       'ltkOri', 'ltkWNB', 'likWCe', 'likWBB'],
      dtype='object')

In [41]:
column = 'ldkPer'
used_keys[column]

'perimeter of enclosure'

Specify a column and pass its values into a choropleth representation within the map. 

In [42]:

classifier = classify(enc[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Streets

Load data and ensure the geometries are all valid Polygons.

In [23]:
streets = gpd.read_parquet(f"{chars_dir}streets_chars_{region}.parquet")

streets.geometry = streets.make_valid()

Create a lonboard layer

In [24]:
%%time
layer = lonboard.PathLayer.from_geopandas(streets.to_crs(4326), width_min_pixels=1)

CPU times: user 65.1 ms, sys: 11.9 ms, total: 77 ms
Wall time: 76.6 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [25]:
sc = Sidecar(title='streets')

Create a Map object

In [26]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [27]:
with sc:
    display(m)

List avaialable columns

In [28]:
assert np.allclose(streets['sdsLen'] , streets.geometry.length)

if 'mm_len' in streets.columns:
    assert np.allclose(streets['mm_len'] , streets.geometry.length)

In [29]:
streets.columns

Index(['geometry', '_status', 'mm_len', 'cdsbool', 'node_start', 'node_end',
       'sdsLen', 'sssLin', 'ldsMSL', 'sdsAre', 'ldsRea', 'ldsAre', 'sisBpM',
       'sdsSPW', 'sdsSPO', 'sdsSWD', 'nID'],
      dtype='object')

In [30]:
streets[['sdsLen', 'sssLin', 'ldsMSL', 'sdsAre', 'ldsRea', 'ldsAre',
       'sisBpM', 'sdsSPW', 'sdsSPO', 'sdsSWD']].describe()

,sdsLen,sssLin,ldsMSL,sdsAre,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD
count,17582.000000,17582.000000,17582.000000,1.187700e+04,17582.000000,1.423000e+04,11713.000000,17582.000000,17582.000000,12361.000000
mean,191.834685,0.964771,197.526050,2.049231e+04,204.855762,3.538634e+05,0.093414,33.790329,0.695741,3.689843
std,476.225311,0.103972,191.610393,5.465022e+04,217.551898,4.066425e+05,0.127457,13.083282,0.267717,1.938601
min,1.297967,0.000000,1.440465,8.644818e-01,0.000000,8.644818e-01,0.000140,0.000000,0.000000,0.000000
25%,57.746118,0.987424,109.545649,3.233232e+03,42.000000,1.516750e+05,0.035203,22.096769,0.471429,2.335431
50%,101.640103,0.999802,141.483531,8.003987e+03,165.000000,2.542413e+05,0.070632,32.338035,0.700000,3.765587
75%,185.700717,1.000000,200.404856,1.943060e+04,293.000000,4.213354e+05,0.120036,50.000000,1.000000,5.078759
max,15676.436708,1.000000,4494.794508,2.797767e+06,2959.000000,8.057106e+06,6.000424,50.000000,1.000000,11.422474


Specify a column and pass its values into a choropleth representation within the map. 

In [31]:
column = 'sisBpM'
used_keys[column]

'buildings per meter of street segment'

In [32]:
streets[column] = streets[column].fillna(0)

In [33]:
classifier = classify(streets[column].astype(int), 'equalinterval', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Nodes

Load data and ensure the geometries are all valid Polygons.

In [151]:
nodes = gpd.read_parquet(f"{chars_dir}nodes_chars_{region}.parquet").to_crs(4326)

Create a lonboard layer

In [152]:
%%time
layer = lonboard.ScatterplotLayer.from_geopandas(nodes, radius_min_pixels=2)

CPU times: user 69.3 ms, sys: 4.83 ms, total: 74.2 ms
Wall time: 73.6 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [153]:
sc = Sidecar(title='nodes')

Create a Map object

In [154]:
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)

Display map within the sidecar plugin

In [155]:
with sc:
    display(m)

List avaialable columns

In [156]:
nodes.columns

Index(['x', 'y', 'mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'nodeID', 'geometry',
       'sddAre', 'midRea', 'midAre'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [161]:
nodes[['mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'nodeID', 'geometry',
       'sddAre', 'midRea', 'midAre']].describe()

,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,nodeID,sddAre,midRea,midAre
count,61838.000000,61838.000000,61838.000000,61838.000000,61838.000000,6.183800e+04,61838.000000,61838.000000,61838.000000,61838.000000,61838.000000,61838.00000,4.654500e+04,61838.000000,4.908000e+04
mean,2.567418,0.105057,0.683189,0.130161,0.183928,1.004460e-06,0.005973,0.009984,319.454293,0.027892,183.852532,30918.50000,2.587954e+04,20.845176,6.426385e+04
std,1.005752,0.060368,0.116292,0.098865,0.115463,6.709700e-07,0.007673,0.005341,612.723878,0.053212,280.458199,17851.23731,4.584542e+04,23.542853,8.924184e+04
min,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000396,0.000000,0.000000,0.000000,2.311779,0.00000,1.159084e+00,0.000000,1.221930e+01
25%,1.000000,0.061856,0.625000,0.058824,0.111111,4.621507e-07,0.003492,0.005882,73.066534,0.000000,74.482961,15459.25000,5.936987e+03,3.000000,1.793014e+04
50%,3.000000,0.098592,0.695652,0.111111,0.169492,8.682536e-07,0.005624,0.009957,181.984598,0.000000,109.536100,30918.50000,1.170070e+04,16.000000,3.585703e+04
75%,3.000000,0.142857,0.757576,0.181818,0.238095,1.420323e-06,0.007255,0.013344,364.331816,0.047619,170.624712,46377.75000,2.549037e+04,31.000000,7.125945e+04
max,6.000000,0.800000,1.000000,0.777778,1.000000,7.283451e-06,0.534588,0.119944,22877.000260,1.000000,11193.902019,61837.00000,1.170909e+06,583.000000,1.806340e+06


In [185]:
column = 'mtdMDi'
used_keys[column]

'mean distance to neighbouring nodes of street network'

In [186]:

classifier = classify(nodes[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])